

## Model 2 Summary

### Features
- RestingBP  
- Cholesterol  
- RestingECG  

### Model
- Logistic Regression  
- Threshold adjusted to **0.40** to prioritize recall  

### Performance
- ROC-AUC: **0.65**  
- Recall: **0.88**  
- Escalation rate: **87%**  

### Interpretation
- Moderate ability to rank patient risk  
- High recall achieved through threshold tuning  
- Limited filtering capacity (majority of patients escalated)  
- Suitable only as an intermediate screening step, not final decision-making model  

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [6]:
df = pd.read_csv("../Data/Raw/heart_disease.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [7]:
df.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='str')

In [8]:
df.dtypes

Age                 int64
Sex                   str
ChestPainType         str
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG            str
MaxHR               int64
ExerciseAngina        str
Oldpeak           float64
ST_Slope              str
HeartDisease        int64
dtype: object

In [9]:
df.shape

(918, 12)

In [10]:
df.isnull().sum()


Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [11]:
df['HeartDisease'].value_counts()
df['HeartDisease'].value_counts(normalize=True)

HeartDisease
1    0.553377
0    0.446623
Name: proportion, dtype: float64

## **Train/Test Split**

x = RestingBP, Cholesterol, RestingECG
y = HeartDisease

In [12]:
X = df[['RestingBP', 'Cholesterol', 'RestingECG']]
y = df['HeartDisease']
X.head()

,RestingBP,Cholesterol,RestingECG
0,140,289,Normal
1,160,180,Normal
2,130,283,ST
3,138,214,Normal
4,150,195,Normal


In [13]:
y.head()

0    0
1    1
2    0
3    1
4    0
Name: HeartDisease, dtype: int64

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state = 42,
    stratify = y
)

In [15]:
y_train.value_counts(normalize=True)


HeartDisease
1    0.553134
0    0.446866
Name: proportion, dtype: float64

In [16]:
y_test.value_counts(normalize=True)

HeartDisease
1    0.554348
0    0.445652
Name: proportion, dtype: float64

1. Scales numeric features
2. One-hot encodes categorical feature
3. Combines them correctly

## Scales numeric features


In [17]:
# grouping the train set columns by type

num_features = ['RestingBP', 'Cholesterol']
cat_features = ['RestingECG']

In [18]:
#preprocessing

preprocess = ColumnTransformer(
    transformers =[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

In [19]:
# model

model = LogisticRegression(max_iter=1000)

In [20]:
# full pipeline: preprocess -> model

clf = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', model)
])

In [21]:
clf.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [22]:
y_proba = clf.predict_proba(X_test)[:, 1]

In [23]:
# risk scores. 
# Predicted probabilities that HeartDisease = 1

y_proba[:10]

array([0.71001231, 0.59220072, 0.52303838, 0.49184025, 0.51363733,
       0.42332985, 0.5187517 , 0.69507614, 0.45611579, 0.52905615])

### To increase recall we need to change the threshold.

Right now it is at 0.5. which mean that people with a 0.49 of probability will not be "labelt" as a risk, which can turn out problematic.

In [24]:
y_pred_default = (y_proba >= 0.5).astype(int)

print(confusion_matrix(y_test, y_pred_default))
print(classification_report(y_test, y_pred_default))

[[32 50]
 [29 73]]
              precision    recall  f1-score   support

           0       0.52      0.39      0.45        82
           1       0.59      0.72      0.65       102

    accuracy                           0.57       184
   macro avg       0.56      0.55      0.55       184
weighted avg       0.56      0.57      0.56       184



In [25]:
y_pred_03 = (y_proba>=0.3).astype(int)

print(confusion_matrix(y_test, y_pred_03))
print(classification_report(y_test, y_pred_03))

[[  1  81]
 [  2 100]]
              precision    recall  f1-score   support

           0       0.33      0.01      0.02        82
           1       0.55      0.98      0.71       102

    accuracy                           0.55       184
   macro avg       0.44      0.50      0.37       184
weighted avg       0.45      0.55      0.40       184





Right now:

At 0.5:
Recall = 0.72
FN = 29

At 0.3:
Recall = 0.98
FN = 2
FP = 81

We need something in between.

(Gotta say, difficult to unnderstand where the alance is supose to be here. Holdeing in mind that this is about peoples lives and on the other hand cost and efficiency. 0.5 is to low but at 0.3 we would almost send everyone for the next expensive test which 81 (FP) did not need. Which will also make longer waiting list everyhwere.)

In [26]:
for t in [0.45, 0.4, 0.35]:
    y_pred_t = (y_proba >= t).astype(int)
    cm = confusion_matrix(y_test, y_pred_t)
    print(f"Threshold: {t}")
    print(cm)
    print(classification_report(y_test, y_pred_t, digits=3))

Threshold: 0.45
[[18 64]
 [17 85]]
              precision    recall  f1-score   support

           0      0.514     0.220     0.308        82
           1      0.570     0.833     0.677       102

    accuracy                          0.560       184
   macro avg      0.542     0.526     0.492       184
weighted avg      0.545     0.560     0.513       184

Threshold: 0.4
[[12 70]
 [12 90]]
              precision    recall  f1-score   support

           0      0.500     0.146     0.226        82
           1      0.562     0.882     0.687       102

    accuracy                          0.554       184
   macro avg      0.531     0.514     0.457       184
weighted avg      0.535     0.554     0.482       184

Threshold: 0.35
[[ 3 79]
 [ 6 96]]
              precision    recall  f1-score   support

           0      0.333     0.037     0.066        82
           1      0.549     0.941     0.693       102

    accuracy                          0.538       184
   macro avg      0.441 

In [27]:
y_pred_035 = (y_proba>=0.35).astype(int)

print(confusion_matrix(y_test, y_pred_035))
print(classification_report(y_test, y_pred_035))

[[ 3 79]
 [ 6 96]]
              precision    recall  f1-score   support

           0       0.33      0.04      0.07        82
           1       0.55      0.94      0.69       102

    accuracy                           0.54       184
   macro avg       0.44      0.49      0.38       184
weighted avg       0.45      0.54      0.41       184



### Picking a Threshold

Model 2 decides:
Who goes to the expensive ETT test.

With 0.3 and 0.35 almost everyone goes to the ETT. The model losses its filter function.

With 0.5 the model misses to many sick people.

with 0.4 and 0.45 is the trade-of the smalles.
From 0.45 → 0.40:
FN: 17 → 12 (5 fewer missed patients)
FP: 64 → 70 (6 extra false positives)
That’s a very balanced trade.

## **Moving forward with the 0.40**
Why?
- Recall = 88%
- Only 12 missed sick patients
- Still filtering somewhat (not sending literally everyone forward)

It shows:
- You prioritized recall (medical context)
- But you did not destroy precision entirely
- The model still behaves like a screening step

Justification for Threshold = 0.40

We selected a decision threshold of 0.40 because it provides a strong balance between sensitivity and resource control.

At 0.40:

- Recall is 88%, meaning we correctly identify most patients with heart disease.
- False negatives are reduced to 12, significantly lower than at the default 0.50 threshold.
- False positives increase moderately, but not to the extreme level observed at lower thresholds such as 0.30 or 0.35.

Since Model 2 represents a mid-stage screening step before an expensive ETT test, the goal is to prioritize recall while still maintaining a meaningful filtering function.

A lower threshold (e.g., 0.30) would send nearly all patients to the next stage, eliminating the purpose of this model as a screening layer.

Therefore, 0.40 provides a medically responsible compromise:
- High sensitivity
- Controlled escalation to the next diagnostic step

In [28]:
threshold = 0.40
y_pred_final = (y_proba >= threshold).astype(int)

In [29]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_final))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_final, digits=3))

print("\nROC-AUC:", roc_auc_score(y_test, y_proba))

Confusion Matrix:
[[12 70]
 [12 90]]

Classification Report:
              precision    recall  f1-score   support

           0      0.500     0.146     0.226        82
           1      0.562     0.882     0.687       102

    accuracy                          0.554       184
   macro avg      0.531     0.514     0.457       184
weighted avg      0.535     0.554     0.482       184


ROC-AUC: 0.6510640841702534


In [30]:
escalation_rate = np.mean(y_pred_final)
print("Escalation rate:", escalation_rate)

Escalation rate: 0.8695652173913043


Limited model because of the lack of features. (model 3 will be very interesting) 

ROC is quite low. And escalation rate on the high side. 
meaning this model in weak and does not filter that much.

#### Cross validation

In [31]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

auc_scores = cross_val_score(clf, X, y, cv=cv, scoring="roc_auc")

print("Fold ROC-AUC scores:", auc_scores)
print("Mean ROC-AUC:", auc_scores.mean())
print("std ROC-AUC:", auc_scores.std())
      

Fold ROC-AUC scores: [0.67521521 0.62523912 0.60084888 0.67429365 0.69904612]
Mean ROC-AUC: 0.6549285954420405
std ROC-AUC: 0.036163487862937965


#### No different outcome with the Cross validation.

The model is still Limited because of the lack of features.
But it is stable, predicable and not overfitting